# Rutas y modulos

In [ ]:
# Importar librerías necesarias
import os  # Para trabajar con rutas del sistema
import pandas as pd  # Para trabajar con dataframes y archivos Excel
import numpy as np  # Para operaciones numéricas
import re  # Para validación de correos electrónicos
from datetime import datetime  # Para trabajar con fechas
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from datetime import datetime
import sys  # Acceso a variables y funciones del sistema

import os
import pandas as pd
import numpy as np
import re
from datetime import datetime
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import sys
from pathlib import Path


## DETECCIÓN AUTOMÁTICA DEL PROYECTO

In [ ]:
# ========================
# 🔧 DETECCIÓN AUTOMÁTICA DEL PROYECTO
# ========================
# Detectar la raíz del proyecto (donde está la carpeta 'src')
notebook_path = Path.cwd()
project_root = notebook_path

while not (project_root / 'src').exists() and project_root != project_root.parent:
    project_root = project_root.parent

if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))
    print(f"✅ Proyecto detectado: {project_root}")

# Importar módulos personalizados
try:
    from src.file_loader import cargar_maestros_ADRES
    from src.data_cleaning import BduaReportProcessor, DataCleaner
    print("✅ Módulos importados correctamente")
except ImportError as e:
    print(f"❌ Error: {e}")

## 🚩 CONFIGURACIÓN (SOLO EDITAR AQUÍ)

In [ ]:
# ========================
# 📅 CONFIGURACIÓN (SOLO EDITAR AQUÍ)
# ========================
# Fecha principal del proceso
Fecha = "10-02-2026"  # Formato: DD-MM-YYYY

# 📅 Fechas específicas de archivos (editar según disponibilidad)
FECHAS_ARCHIVOS = {
    "maestros_adres": "06-02-2026",      # Fecha de corte de maestros ADRES
    "ms_sie": None, #"03-02-2026",              # Fecha de reporte MS del SIE
    "pila_movilidad": None,              # None = usa fecha principal
}

# Información del informe CTO (editar según corresponda)
INFORME_CTO = {
    "codigo": "CTO 102.2026",
    "numero": "#02",
    "actividad": "12 Actividad",
    "subcarpeta": "Bases de datos notificaciones telefonicas/MOVILIDAD NO EFECTIVA"
}

# Si NO usas estructura CTO, usa esto:
# INFORME_CTO = None

## CONSTRUCCIÓN AUTOMÁTICA DE FECHAS

In [ ]:
# ========================
# 🗓️ CONSTRUCCIÓN AUTOMÁTICA DE FECHAS
# ========================
def parsear_fecha(fecha_str):
    """Convierte fecha DD-MM-YYYY a componentes individuales"""
    fecha_obj = datetime.strptime(fecha_str, "%d-%m-%Y")
    return {
        "obj": fecha_obj,
        "año": str(fecha_obj.year),
        "mes_numero": f"{fecha_obj.month:02d}",
        "dia": f"{fecha_obj.day:02d}",
        "mes_nombre": {
            1:"Enero", 2:"Febrero", 3:"Marzo", 4:"Abril", 5:"Mayo", 6:"Junio",
            7:"Julio", 8:"Agosto", 9:"Septiembre", 10:"Octubre", 11:"Noviembre", 12:"Diciembre"
        }[fecha_obj.month],
        "fecha_str": fecha_str
    }

# Parsear fecha principal
fecha_principal = parsear_fecha(Fecha)
año = fecha_principal["año"]
mes_numero = fecha_principal["mes_numero"]
dia = fecha_principal["dia"]
mes_nombre = fecha_principal["mes_nombre"]

# Parsear fechas de archivos (usa fecha principal si no está especificada)
fecha_maestros = parsear_fecha(FECHAS_ARCHIVOS.get("maestros_adres") or Fecha)
fecha_sie = parsear_fecha(FECHAS_ARCHIVOS.get("ms_sie") or Fecha)
fecha_pila = parsear_fecha(FECHAS_ARCHIVOS.get("pila_movilidad") or Fecha)

# Nombres de archivos con fechas específicas
ARCHIVOS = {
    "maestro_contributivo": f"EPSC25MC00{fecha_maestros['dia']}{fecha_maestros['mes_numero']}{fecha_maestros['año']}.TXT",
    "maestro_subsidiado": f"EPS025MS00{fecha_maestros['dia']}{fecha_maestros['mes_numero']}{fecha_maestros['año']}.TXT",
    "ms_sie": f"Reporte_Validación Archivos Maestro_{fecha_sie['año']}_{fecha_sie['mes_numero']}_{fecha_sie['dia']}.csv",
    "pila_movilidad": f"Dataframe Pila {fecha_pila['fecha_str']}.xlsx",
    "cod_dane": "Departamentos.txt",
    "salida_excel": f"Movilidad No Efectiva {Fecha}.xlsx"
}

# Hoja de Excel
hoja = "Sheet1"

# ========================
# 📂 CONSTRUCCIÓN AUTOMÁTICA DE RUTAS
# ========================
# Detectar usuario automáticamente
usuario = os.getlogin()
print(f"👤 Usuario detectado: {usuario}")

# Base OneDrive (estándar para todos)
onedrive_base = Path(f"C:/Users/{usuario}/OneDrive - 891856000_CAPRESOCA E P S")

# Verificar OneDrive
if not onedrive_base.exists():
    print(f"⚠️  Advertencia: OneDrive no encontrado en: {onedrive_base}")

# ========================
# 🗂️ RUTAS DE ARCHIVOS DE ENTRADA
# ========================
# Base Capresoca
capresoca_base = onedrive_base / "Capresoca/AlmostClear"

# Maestros ADRES (usa su fecha específica para la carpeta del año)
R_Maestro__EPSC25 = capresoca_base / "Procesos BDUA/Contributivo/Maestro" / fecha_maestros['año'] / ARCHIVOS["maestro_contributivo"]
R_Maestro__EPS025 = capresoca_base / "Procesos BDUA/Subsidiados/Maestro/MS" / fecha_maestros['año'] / ARCHIVOS["maestro_subsidiado"]

# MS del SIE
R_MS_SIe = capresoca_base / "SIE/Aseguramiento/ms_sie" / ARCHIVOS["ms_sie"]

# PILA Movilidad (usa la carpeta de la fecha principal del proceso)
traslados_base = onedrive_base / "Escritorio/Yesid Rincón Z/Traslados/Procesos BDUA"
ruta_proceso = traslados_base / año / f"{mes_numero}_{mes_nombre}" / dia
R_Pila_movilidad = ruta_proceso / ARCHIVOS["pila_movilidad"]

# Código DANE
R_Cod_DANE = capresoca_base / "Constantes" / ARCHIVOS["cod_dane"]

# ========================
# 📤 RUTA DE SALIDA
# ========================
if INFORME_CTO:
    # Ruta para estructura de informes CTO
    informes_base = onedrive_base / "Escritorio/Yesid Rincón Z/informes"
    codigo_limpio = INFORME_CTO["codigo"].replace(' ', '')
    informe_carpeta = f"{codigo_limpio} Informe  {INFORME_CTO['numero']}"
    
    S_Excel = (informes_base / año / INFORME_CTO["codigo"] / informe_carpeta / 
               INFORME_CTO["actividad"] / INFORME_CTO["subcarpeta"] / ARCHIVOS["salida_excel"])
else:
    S_Excel = ruta_proceso / ARCHIVOS["salida_excel"]

Ruta_Salida = S_Excel.parent

# ========================
# ✅ VERIFICACIÓN DE ARCHIVOS
# ========================
print(f"\n{'='*70}")
print(f"🔍 VERIFICACIÓN DE RUTAS Y ARCHIVOS")
print(f"{'='*70}")
print(f"👤 Usuario: {usuario}")
print(f"📅 Fecha Principal: {Fecha} ({mes_nombre} {año})")
print(f"📅 Fecha Maestros ADRES: {fecha_maestros['fecha_str']}")
print(f"📅 Fecha MS SIE: {fecha_sie['fecha_str']}")
print(f"📅 Fecha PILA Movilidad: {fecha_pila['fecha_str']}")
print(f"📁 Proyecto: {project_root}")
print(f"📁 OneDrive: {onedrive_base}")

if INFORME_CTO:
    print(f"📋 Informe CTO: {INFORME_CTO['codigo']} {INFORME_CTO['numero']}")
    print(f"📁 Actividad: {INFORME_CTO['actividad']}")

print(f"📁 Ruta Salida: {Ruta_Salida}")

rutas_verificar = {
    "📊 Maestro Contributivo": R_Maestro__EPSC25,
    "📊 Maestro Subsidiado": R_Maestro__EPS025,
    "📋 MS SIE": R_MS_SIe,
    "📄 PILA Movilidad": R_Pila_movilidad,
    "🗺️  Código DANE": R_Cod_DANE
}

print(f"\n📄 Archivos de entrada:")
todas_existen = True
for nombre, ruta in rutas_verificar.items():
    existe = ruta.exists()
    simbolo = "✅" if existe else "❌"
    todas_existen = todas_existen and existe
    print(f"  {simbolo} {nombre}")
    print(f"     └─ {ruta.name}")
    if not existe:
        print(f"     └─ ⚠️  {ruta}")

print(f"\n📤 Archivo de salida:")
print(f"  📁 {S_Excel.name}")
print(f"     └─ Directorio: {Ruta_Salida}")
if not Ruta_Salida.exists():
    print(f"     └─ ⚠️  El directorio de salida no existe, se creará al guardar")

print(f"{'='*70}\n")

if not todas_existen:
    print("⚠️  ADVERTENCIA: Algunos archivos no existen")
else:
    print("✅ Todos los archivos verificados\n")

# ========================
# 🎯 RESUMEN DE CONFIGURACIÓN
# ========================
print("🎯 Configuración activa:")
print(f"   📅 Fecha proceso: {Fecha}")
print(f"   📅 Maestros ADRES: {fecha_maestros['fecha_str']} → {ARCHIVOS['maestro_contributivo']}")
print(f"   📅 MS SIE: {fecha_sie['fecha_str']} → {ARCHIVOS['ms_sie']}")
print(f"   📅 PILA: {fecha_pila['fecha_str']} → {ARCHIVOS['pila_movilidad']}")
print(f"   👤 Usuario: {usuario}")
print(f"   📁 Hoja Excel: {hoja}")
print(f"\n📊 Variables de rutas disponibles:")
print(f"   - R_Maestro__EPSC25")
print(f"   - R_Maestro__EPS025")
print(f"   - R_MS_SIe")
print(f"   - R_Pila_movilidad")
print(f"   - R_Cod_DANE")
print(f"   - S_Excel")
print(f"   - Ruta_Salida\n")

# Cargue Dataframe

In [ ]:
# Cargar y combinar los maestros
maestro_ADRES = cargar_maestros_ADRES(R_Maestro__EPS025, R_Maestro__EPSC25)

# Duplicar la columna "MARCASISBENIV+MARCASISBENIII_2" y nombrarla "MARCASISBENIV+MARCASISBENIII"
maestro_ADRES["MARCASISBENIV+MARCASISBENIII_2"] = \
    maestro_ADRES["MARCASISBENIV+MARCASISBENIII"]


# 1. Instanciar el procesador: Se crea un objeto pasando el DataFrame.
#    La jerarquía de población ya está definida por defecto dentro de la clase.
processor = BduaReportProcessor(df=maestro_ADRES)

# 2. Ejecutar la limpieza y asignarla de vuelta.
#    El método retorna un DataFrame completamente nuevo con la columna actualizada.
maestro_ADRES = processor.prioritize_population_markers(
    col_name="MARCASISBENIV+MARCASISBENIII"
)

# ¡Listo! 'maestro_ADRES' ahora contiene los datos limpios.

In [ ]:
# Cargar el archivo Excel R_Pila_movilidad en un DataFrame
df_pila_movilidad = pd.read_excel(R_Pila_movilidad, sheet_name=hoja, dtype=str)
df_ms_SIE = pd.read_csv(R_MS_SIe, sep=';', encoding='ansi', dtype=str)
df_cod_dane = pd.read_csv(R_Cod_DANE, sep=';', encoding='UTF-8', dtype=str)

# Normalizar datos

In [ ]:
print(df_pila_movilidad['Movilidad'].unique())

In [ ]:
# Filtrar registros donde 'Movilidad' contenga 'sin sisben' o 'sisben D' (ignorando mayúsculas/minúsculas)
filtro = df_pila_movilidad['Movilidad'].str.contains('sin sisben', case=False, na=False) | \
         df_pila_movilidad['Movilidad'].str.contains('no sisben', case=False, na=False) | \
         df_pila_movilidad['Movilidad'].str.contains('sisben D', case=False, na=False)

# Mostrar cantidad antes del filtrado
print(f"Registros antes del filtrado: {len(df_pila_movilidad)}")

# Aplicar el filtro
df_pila_movilidad = df_pila_movilidad[filtro]

# Mostrar cantidad después del filtrado
print(f"Registros después del filtrado: {len(df_pila_movilidad)}")

In [ ]:
df_pila_movilidad = df_pila_movilidad[["Tipo Documento Cotizante", "N° Identificación Cotizante", "Población", "Movilidad"]]

In [ ]:
# Asegurar que los identificadores sean string para evitar problemas de merge
maestro_ADRES['TPS_IDN_ID'] = maestro_ADRES['TPS_IDN_ID'].astype(str)
maestro_ADRES['HST_IDN_NUMERO_IDENTIFICACION'] = maestro_ADRES['HST_IDN_NUMERO_IDENTIFICACION'].astype(str)
df_pila_movilidad['Tipo Documento Cotizante'] = df_pila_movilidad['Tipo Documento Cotizante'].astype(str)
df_pila_movilidad['N° Identificación Cotizante'] = df_pila_movilidad['N° Identificación Cotizante'].astype(str)

# Seleccionar columnas a traer del maestro
cols_maestro = [
    "TPS_IDN_ID", "HST_IDN_NUMERO_IDENTIFICACION",
    "AFL_PRIMER_APELLIDO", "AFL_SEGUNDO_APELLIDO",
    "AFL_PRIMER_NOMBRE", "AFL_SEGUNDO_NOMBRE",
    "AFL_FECHA_NACIMIENTO", "TPS_GNR_ID", "DPR_ID", "MNC_ID"
]

# Hacer el merge
df_pila_movilidad = df_pila_movilidad.merge(
    maestro_ADRES[cols_maestro],
    left_on=["Tipo Documento Cotizante", "N° Identificación Cotizante"],
    right_on=["TPS_IDN_ID", "HST_IDN_NUMERO_IDENTIFICACION"],
    how="left"
)

In [ ]:
df_pila_movilidad = df_pila_movilidad.drop(columns=["Tipo Documento Cotizante", "N° Identificación Cotizante"])

In [ ]:
# Seleccionar columnas a unir desde df_ms_SIE
cols_contacto = [
    "tipo_documento", "numero_identificacion",
    "celular", "telefono_1", "telefono_2", "correo_electronico"
]

# Realizar el merge usando los identificadores correspondientes
df_pila_movilidad = df_pila_movilidad.merge(
    df_ms_SIE[cols_contacto],
    left_on=["TPS_IDN_ID", "HST_IDN_NUMERO_IDENTIFICACION"],
    right_on=["tipo_documento", "numero_identificacion"],
    how="left"
)

# Eliminar las columnas de identificadores provenientes de df_ms_SIE
df_pila_movilidad = df_pila_movilidad.drop(columns=["tipo_documento", "numero_identificacion"])

In [ ]:
def limpiar_telefono(numero):
    """
    Limpia y valida un número de teléfono colombiano.
    - Elimina caracteres no numéricos.
    - Debe tener 10 dígitos y empezar por '3'.
    - Si no cumple, retorna ''.
    """
    if pd.isnull(numero):
        return ''
    # Eliminar espacios, signos y dejar solo números
    solo_numeros = re.sub(r'\D', '', str(numero))
    # Validar longitud y prefijo
    if len(solo_numeros) == 10 and solo_numeros.startswith('3'):
        return solo_numeros
    return ''

# Aplicar la función a las columnas de teléfono
for col in ["celular", "telefono_1", "telefono_2"]:
    df_pila_movilidad[col] = df_pila_movilidad[col].apply(limpiar_telefono)

In [ ]:
def limpiar_correo(correo):
    """
    Limpia y valida un correo electrónico:
    - Elimina espacios y convierte a minúsculas.
    - Descarta correos con palabras clave como 'actualizar', 'notiene', 'sincorreo', etc.
    - Valida la estructura básica de correo electrónico.
    - Si no es válido, retorna ''.
    """
    if pd.isnull(correo):
        return ''
    correo = str(correo).strip().lower()
    # Palabras clave no válidas
    palabras_invalidas = [
        'actualizar', 'notiene', 'no tiene', 'sincorreo', 'sin correo', 'noaplica', 'no aplica', 'ninguno', 'noexiste', 'no existe'
    ]
    for palabra in palabras_invalidas:
        if palabra in correo:
            return ''
    # Validar estructura básica de correo
    patron = r'^[\w\.-]+@[\w\.-]+\.\w+$'
    if re.match(patron, correo):
        return correo
    return ''

# Aplicar la función a la columna 'correo_electronico'
df_pila_movilidad['correo_electronico'] = df_pila_movilidad['correo_electronico'].apply(limpiar_correo)

In [ ]:
# Realizar el merge para agregar los nombres de departamento y municipio
df_pila_movilidad = df_pila_movilidad.merge(
    df_cod_dane[['Código Deapartamento', 'Código Municipio', 'Nombre Departamento', 'Nombre Municipio']],
    left_on=['DPR_ID', 'MNC_ID'],
    right_on=['Código Deapartamento', 'Código Municipio'],
    how='left'
)

# Eliminar las columnas de id provenientes de df_cod_dane
df_pila_movilidad = df_pila_movilidad.drop(columns=['Código Deapartamento', 'Código Municipio'])

# Guardar dataframes

In [ ]:
df_pila_movilidad.columns

In [ ]:
with pd.ExcelWriter(S_Excel, engine='openpyxl') as writer:
    for movilidad, grupo in df_pila_movilidad.groupby('Movilidad'):
        # Limpiar el nombre de la hoja para evitar caracteres no permitidos
        nombre_hoja = str(movilidad)[:31].replace('/', '_').replace('\\', '_').replace('*', '_').replace('?', '_').replace('[', '_').replace(']', '_')
        grupo.to_excel(writer, sheet_name=nombre_hoja, index=False)